In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
sample_submission = pd.read_csv("/kaggle/input/tabular-playground-series-oct-2021/sample_submission.csv")
train = pd.read_csv("/kaggle/input/tabular-playground-series-oct-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-oct-2021/test.csv")

In [7]:
sample_submission.head()

In [8]:
train.head()

In [9]:
test.head()

In [10]:
train_null_cols = [col for col in train.iloc[ : , 1 : ].columns if train[col].isnull().sum() != 0]
train_null_cols

In [11]:
test_null_cols = [col for col in test.iloc[ : , 1 : ].columns if test[col].isnull().sum() != 0]
test_null_cols

In [12]:
train.info()

In [19]:
train_int_cols = [col for col in train.iloc[ : , 1 : ].columns if train[col].dtype == "int64"]
len(train_int_cols)

In [21]:
train_float_cols = [col for col in train.iloc[ : , 1 : ].columns if train[col].dtype == "float64"]
len(train_float_cols)

In [15]:
test.info()

In [22]:
test_int_cols = [col for col in test.iloc[ : , 1 : ].columns if test[col].dtype == "int64"]
len(test_int_cols)

In [23]:
test_float_cols = [col for col in test.iloc[ : , 1 : ].columns if test[col].dtype == "float64"]
len(test_float_cols)

In [36]:
train_int_unique = [train[col].unique() for col in train[train_int_cols].columns]
train_int_unique

In [37]:
test_int_unique = [test[col].unique() for col in test[test_int_cols].columns]
test_int_unique

In [24]:
test[test_float_cols]

In [ ]:
test[test_int_cols]

In [ ]:
a = 'id'

test(~test[a])